# The BigFlow tutorial part 1

### What you will learn in part 1
* Making queries
* Writing to tables
* Creating tables
* Creating workflows

In [ ]:
!pip3 install bigflow==0.4.0

Depending on your machine configuration it can also be:

In [ ]:
!pip install bigflow==0.4.0

In [ ]:
import bigflow as bf
import pydata_google_auth

### Task definition
To guide you through the most important features that BigFlow provides, we've prepared a simple task. There is a **transactions** table, which looks like this:

| user_id | transaction_value | partition_timestamp |
|---------|-------------------|---------------------|
| john123 | 800               | 2019-01-01 00:00:00 |
| smith99 | 10000             | 2019-01-01 00:00:00 |
| smith99 | 30000             | 2019-01-01 00:00:00 |

The table contains all transactions that users make on a specific day. Your task is to calculate two metrics for each user:
 daily user transaction value and daily user transaction count.

Final result should be a table named **user_transaction_metrics**:

| user_id | metric_value | metric_name            | partition_timestamp |
|---------|--------------|------------------------|---------------------|
| john123 | 800          | USER_TRANSACTION_VALUE | 2019-01-01 00:00:00 |
| smith99 | 40000        | USER_TRANSACTION_VALUE | 2019-01-01 00:00:00 |
| john123 | 1            | USER_TRANSACTION_COUNT | 2019-01-01 00:00:00 |
| smith99 | 2            | USER_TRANSACTION_COUNT | 2019-01-01 00:00:00 |

### Preparing transactions table
Before you start processing the transactions table, you need to create it. Execute the cell below to create the transactions table.

In [ ]:
PROJECT_ID = ''  # put your own project ID

# If you are using this code in jupyter lab environment you should not use these credentials but service account and set below variable credentials to None.
# If you are using this code on your local machine you should leave credentials variable as it is.

# You will be asked to copy the url to your browser and then login using your credentials. Then copy a token and paste it in jupyter.
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/bigquery'])  

dataset = bf.Dataset(project_id=PROJECT_ID, dataset_name='transactions', credentials=credentials)
dataset.create_table("""
CREATE TABLE IF NOT EXISTS transactions (
    user_id STRING,
    transaction_value FLOAT64,
    partition_timestamp TIMESTAMP)
PARTITION BY DATE(partition_timestamp)""").run()

dataset.write_truncate('transactions', """
SELECT 'john123' as user_id, 800.0 as transaction_value, TIMESTAMP('2019-01-01') as partition_timestamp
""").run('2019-01-01')
dataset.write_append('transactions', """
SELECT 'smith99' as user_id, 10000.0 as transaction_value, TIMESTAMP('2019-01-01') as partition_timestamp
""").run('2019-01-01')
dataset.write_append('transactions', """
SELECT 'smith99' as user_id, 30000.0 as transaction_value, TIMESTAMP('2019-01-01') as partition_timestamp
""").run('2019-01-01')

### The dataset object
Using the dataset object you can manipulate tables inside the specified dataset. You need to put names of the tables that you want to manipulate into the `internal_tables` parameter. The `external_tables` parameter lets you access external tables inside your queries using an alias.

In [ ]:
dataset = bf.Dataset(
    project_id=PROJECT_ID,
    dataset_name='transaction_aggregates',
    internal_tables=['user_transaction_metrics'],
    external_tables={
        'transactions': '{}.transactions.transactions'.format(PROJECT_ID)
    })

### The create_table method
To store the metrics, you need to create the **user_transaction_metrics** table. The `create_table` method returns the component that you can run.

In [ ]:
create_user_transaction_metrics_table = dataset.create_table('''
CREATE TABLE IF NOT EXISTS user_transaction_metrics (
    user_id STRING,
    metric_name STRING,
    metric_value STRING,
    
    partition_timestamp TIMESTAMP)
PARTITION BY DATE(partition_timestamp)
''')

In [ ]:
create_user_transaction_metrics_table.run()

### Simple queries
Take a look at the transactions. You can do that using the `collect` method. The `collect` method returns a component that you can `peek`.
The `peek` method returns the query result in a DataFrame object.

In every operation, you can access run time using `dt` alias and all the tables you've specified in your dataset object.

In [ ]:
all_transactions_from_single_day = dataset.collect('''
SELECT *
FROM `{transactions}`
WHERE DATE(partition_timestamp) = '{dt}'
''')

In [ ]:
all_transactions_from_single_day.peek('2019-01-01')

### Estimating the cost of a query
The dry_run method returns a string with the estimated number of bytes read by the query and estimated cost of a query in dollars.

In [ ]:
costs = dataset.dry_run('''
        SELECT *
        FROM `{transactions}`
        WHERE DATE(partition_timestamp) = '{dt}'
        ''')
costs.run()

### Calculating the first metric using the write_truncate method
The `write_truncate` method saves the result of the query to the table (to the specified partition) and overrides any old content. Before running the component for real, you can
`peek` the query result.

In [ ]:
write_truncate_user_transaction_value = dataset.write_truncate('user_transaction_metrics', 
'''
SELECT 
    TIMESTAMP('{dt}') as partition_timestamp,
    user_id,
    CAST(sum(CAST(transaction_value as FLOAT64)) as string) as metric_value,
    'USER_TRANSACTION_VALUE' as metric_name
FROM `{transactions}`
WHERE DATE(partition_timestamp) = '{dt}'
GROUP BY user_id
''')

In [ ]:
write_truncate_user_transaction_value.peek('2019-01-01', limit=10)

In [ ]:
write_truncate_user_transaction_value.run('2019-01-01')

### Calculating the second metric using the write_append method
The only difference between the `write_append` and `write_truncate` is that `write_append` does not override old content. You can use that behaviour to add the second metric
to the table.

In [ ]:
write_append_user_transaction_count = dataset.write_append('user_transaction_metrics', 
'''
SELECT 
    TIMESTAMP('{dt}') as partition_timestamp,
    user_id,
    CAST(count(*) as string) as metric_value,
    'USER_TRANSACTION_COUNT' as metric_name
FROM `{transactions}`
WHERE DATE(partition_timestamp) = '{dt}'
GROUP BY user_id
''')

In [ ]:
write_append_user_transaction_count.peek('2019-01-01')

In [ ]:
write_append_user_transaction_count.run('2019-01-01')

In [ ]:
dataset.collect('''
SELECT *
FROM `{user_transaction_metrics}`
WHERE date(partition_timestamp) = '{dt}'
''').peek('2019-01-01')

### Creating the workflow
Finally, create the workflow using components that you created. You can `run` the workflow just like a component. **To deploy your workflow, go to the tutorial part 2**.

In [ ]:
workflow = bf.Workflow(definition=[
    create_user_transaction_metrics_table.to_job(),
    write_truncate_user_transaction_value.to_job(),
    write_append_user_transaction_count.to_job()])

In [ ]:
workflow.run('2019-01-02')